In [13]:
# Translation from eng to french
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as seaborn

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
#from keras.utils.sequences import pad_sequences

from keras.layers import LSTM,Embedding,Input,Dense
from keras.models import Sequential,Model

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
df=pd.read_csv('/content/eng_-french.csv')
df.columns=['English','French']
df.head()

,English,French
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   English  175621 non-null  object
 1   French   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB


In [11]:
data=df[:]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   English  175621 non-null  object
 1   French   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB


# Clean the dataset

In [15]:
def clean_eng(text):
    text=text.lower()
    # Remove all value except a-z, !,?
    text=re.sub(u"[^a-z!?,]"," ",text)
    # Tokenize - word wise
    text=nltk.word_tokenize(text)
    # Join text
    text=" ".join([i.strip() for i in text])
    return text

clean_eng(data.iloc[0,0])

'hi'

In [16]:
data.iloc[1,0], clean_eng(data.iloc[1,0])

('Run!', 'run !')

In [17]:
def clean_french(text):
    text=text.lower()
    # Remove all value except a-z, !,?
    text=re.sub(u"[^a-zéâàçêêëôîû!?,]"," ",text)

    return text

clean_french(data.iloc[0,1])

'salut!'

In [18]:
data.iloc[0,1], clean_eng(data.iloc[0,1])

('Salut!', 'salut !')

In [19]:
data['English']=data['English'].apply(lambda txt:clean_eng(txt))
data['French']=data['French'].apply(lambda txt:clean_french(txt))

In [20]:
data.head()

,English,French
0,hi,salut!
1,run !,cours !
2,run !,courez !
3,who ?,qui ?
4,wow !,ça alors !


In [21]:
# add <start> <end> token to decoder sentence
data['French']=data['French'].apply(lambda txt:f"<start>{txt}<end>")
data

,English,French
0,hi,<start>salut!<end>
1,run !,<start>cours !<end>
2,run !,<start>courez !<end>
3,who ?,<start>qui ?<end>
4,wow !,<start>ça alors !<end>
...,...,...
175616,"top down economics never works , said obama th...",<start> l économie en partant du haut vers le...
175617,a carbon footprint is the amount of carbon dio...,<start>une empreinte carbone est la somme de p...
175618,death is something that we re often discourage...,<start>la mort est une chose qu on nous décour...
175619,since there are usually multiple websites on a...,<start>puisqu il y a de multiples sites web su...


In [23]:
# Tokenization and Build Vocabulary
#English Tokenization
english_tokenize=Tokenizer(filters='#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n')
english_tokenize.fit_on_texts(data['English'])

In [24]:
english_tokenize

In [25]:
num_encoder_tokens=len(english_tokenize.word_index)
num_encoder_tokens

13905

In [27]:
encoder= english_tokenize.texts_to_sequences(data['English'])
encoder[:5]

[[2752], [417, 124], [417, 124], [76, 5], [3489, 124]]

In [28]:
max_encoder_sequence_len=np.max([len(enc) for enc in encoder])
max_encoder_sequence_len

47

In [29]:
#French Tokenization
french_tokenize=Tokenizer(filters='#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n')
french_tokenize.fit_on_texts(data['French'])

In [30]:
num_decoder_tokens=len(french_tokenize.word_index)
num_decoder_tokens

24129

In [31]:
decoder= french_tokenize.texts_to_sequences(data['French'])
decoder[:5]

[[2, 15399, 1],
 [2, 551, 40, 1],
 [2, 4807, 40, 1],
 [2, 46, 6, 1],
 [2, 38, 381, 40, 1]]

In [33]:
max_decoder_sequence_len=np.max([len(dec) for dec in decoder])
max_decoder_sequence_len

61

In [34]:
idx_2_txt_decoder={k:i for i,k in french_tokenize.word_index.items()}
idx_2_txt_decoder[1]

'end'

In [35]:
idx_2_txt_encoder={k:i for i,k in english_tokenize.word_index.items()}
idx_2_txt_encoder[1]

'i'

In [36]:
idx_2_txt_decoder[0]="<pad>"
idx_2_txt_encoder[0]="<pad>"

In [40]:
from nltk.lm.preprocessing import pad_sequence
# Pad sequence is required in both encoder and decoder.

from keras.preprocessing import sequence
from keras.utils import pad_sequences
encoder_seq=pad_sequences(encoder,maxlen=max_encoder_sequence_len,padding='post')
encoder_seq

array([[2752,    0,    0, ...,    0,    0,    0],
       [ 417,  124,    0, ...,    0,    0,    0],
       [ 417,  124,    0, ...,    0,    0,    0],
       ...,
       [ 607,    8,   99, ...,    0,    0,    0],
       [ 361,   47,   27, ...,    0,    0,    0],
       [  65,  276,   76, ...,    6, 1100, 1448]], dtype=int32)

In [41]:
encoder_seq.shape

(175621, 47)

In [42]:
decoder_seq=pad_sequences(decoder,maxlen=max_decoder_sequence_len,padding='post')
decoder_seq

array([[    2, 15399,     1, ...,     0,     0,     0],
       [    2,   551,    40, ...,     0,     0,     0],
       [    2,  4807,    40, ...,     0,     0,     0],
       ...,
       [    2,    14,   327, ...,     0,     0,     0],
       [    2,  7523,    10, ...,     0,     0,     0],
       [    2,    59,   173, ...,  2545,  2136,     1]], dtype=int32)

In [43]:
decoder_seq.shape

(175621, 61)

Build LSTM Model

In [48]:
from keras.layers import Activation,SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

# Encoder model
# Functional API

encoder_input=Input(shape=(None,), name='encoder_input_layer')
encoder_embedding=Embedding(num_encoder_tokens,300,input_length=max_encoder_sequence_len,
                            name='encoder_emedding_layer')(encoder_input)
encoder_lstm=LSTM(256,activation='tanh',return_sequences=True,
                  return_state=True,name='encoder_lstm1_layer')(encoder_embedding)
encoder_lstm2=LSTM(256,activation='tanh',return_sequences=True,
                  return_state=True,name='encoder_lstm2_layer')(encoder_lstm)                

_,state_h,state_c=encoder_lstm2
encoder_states=[state_h,state_c]

In [49]:
# Decoder model
# Functional API

decoder_input=Input(shape=(None,), name='decoder_input_layer')
decoder_embedding=Embedding(num_decoder_tokens,300,input_length=max_decoder_sequence_len,
                            name='decoder_emedding_layer')(decoder_input)
decoder_lstm=LSTM(256,activation='tanh',return_sequences=True,
                  return_state=True,name='decoder_lstm1_layer')
               

decoder_output,_,_=decoder_lstm(decoder_embedding,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens+1,activation='softmax',name='decoder_final_layer')
outputs=decoder_dense(decoder_output)

In [50]:
model=Model([encoder_input, decoder_input],outputs)

In [51]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, None)]      0           []                               
 r)                                                                                               
                                                                                                  
 encoder_emedding_layer (Embedd  (None, None, 300)   4171500     ['encoder_input_layer[0][0]']    
 ing)                                                                                             
                                                                                                  
 decoder_input_layer (InputLaye  [(None, None)]      0           []                               
 r)                                                                                         

In [52]:
model.compile(otimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
history=model.fit(encoder_seq,decoder_seq,epochs=1,batch_size=64)

TypeError: ignored